# Коллаборативная фильтрация

In [ ]:
ПАКЕТ SURPRISE

используйте данные MovieLens 1M
можно использовать любые модели из пакета
получите RMSE на тестовом сете 0.87 и ниже
Комментарий преподавателя :
В ДЗ на датасет 1М может не хватить RAM. Можно сделать на 100K. 
Качество RMSE предлагаю считать на основе CrossValidation (5 фолдов), а не отложенном датасете.

In [1]:
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split

import pandas as pd

In [2]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

In [3]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
movies.shape

(9742, 3)

In [5]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [6]:
ratings.shape

(100836, 4)

In [7]:
movies_with_ratings = movies.join(ratings.set_index('movieId'), on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)

In [8]:
movies_with_ratings.shape

(100836, 6)

In [9]:
dataset = pd.DataFrame({
    'uid': movies_with_ratings.userId,
    'iid': movies_with_ratings.title,
    'rating': movies_with_ratings.rating
})

In [10]:
dataset.head()

,uid,iid,rating
0,1.0,Toy Story (1995),4.0
1,5.0,Toy Story (1995),4.0
2,7.0,Toy Story (1995),4.5
3,15.0,Toy Story (1995),2.5
4,17.0,Toy Story (1995),4.5


In [11]:
reader = Reader(rating_scale=(ratings.rating.min(), ratings.rating.max()))
data = Dataset.load_from_df(dataset, reader)

In [12]:
trainset, testset = train_test_split(data, test_size=0.2)

In [13]:
from surprise import SVD

In [14]:
from surprise.model_selection import cross_validate

In [15]:
algo = SVD()

In [16]:
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8731  0.8780  0.8701  0.8705  0.8734  0.8730  0.0028  
MAE (testset)     0.6681  0.6760  0.6690  0.6668  0.6719  0.6704  0.0033  
Fit time          4.25    4.78    5.32    4.74    5.55    4.93    0.46    
Test time         0.17    0.19    0.12    0.12    0.15    0.15    0.03    


{'test_rmse': array([0.87307716, 0.87795812, 0.87013407, 0.87054975, 0.87336541]),
 'test_mae': array([0.66814551, 0.67599481, 0.66896488, 0.66681097, 0.67185661]),
 'fit_time': (4.2492897510528564,
  4.780309200286865,
  5.323919296264648,
  4.739065885543823,
  5.5492236614227295),
 'test_time': (0.16799473762512207,
  0.19393610954284668,
  0.12419486045837402,
  0.1152048110961914,
  0.14959025382995605)}

In [17]:
algo.fit(trainset)
predictions = algo.test(testset)

In [18]:
accuracy.rmse(predictions)

RMSE: 0.8763


0.876299880759586

In [19]:
from surprise.model_selection import GridSearchCV

In [20]:
param_grid = {'n_factors': [20, 50, 200], 'n_epochs': [20, 50, 200]}

In [21]:
grid = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=5)

In [22]:
grid.fit(data)

In [24]:
grid.best_params['rmse']

{'n_factors': 200, 'n_epochs': 200}

In [23]:
grid.best_score['rmse']

0.8645274570308399